In [1]:
# 10-05 16:20 하이투자증권 - 기술적지표 각 2개

import yfinance as yf
import pandas as pd
import backtrader as bt

# MFI 구현
class MoneyFlowIndex(bt.Indicator):
    lines = ('mfi',)
    params = (('period', 14),)
    
    def __init__(self):
        tp = (self.data.high + self.data.low + self.data.close) / 3.0
        rmf = tp * self.data.volume
        pmf = bt.If(self.data.close > self.data.close(-1), rmf, 0)
        nmf = bt.If(self.data.close < self.data.close(-1), rmf, 0)
        mfr = bt.indicators.SmoothedMovingAverage(pmf, period=self.p.period) / \
              bt.indicators.SmoothedMovingAverage(nmf, period=self.p.period)
        self.lines.mfi = 100.0 - (100.0 / (1.0 + mfr))

# ADI 구현
class ADI(bt.Indicator):
    lines = ('adi_line',)
    
    def __init__(self):
        self.addminperiod(1)  # 최소 기간을 설정합니다. ADI는 이전 데이터를 참조하지 않으므로 1로 설정.

    def next(self):
        close = self.data.close[0]  
        low = self.data.low[0]      
        high = self.data.high[0]    
        volume = self.data.volume[0] 
        adi_value = ((close - low) - (high - close)) / (high - low) * volume
        self.lines.adi_line[0] = adi_value

        
# Accumulation/Distribution Line (ADL) 구현
class ADL(bt.Indicator):
    lines = ('adl_line',)
    
    def __init__(self):
        self.addminperiod(1)

    def next(self):
        close = self.data.close[0]  
        low = self.data.low[0]      
        high = self.data.high[0]    
        volume = self.data.volume[0] 
        
        if high != low:
            adl_value = ((close - low) - (high - close)) / (high - low) * volume
        else:
            adl_value = 0
        
        self.lines.adl_line[0] = adl_value + self.lines.adl_line[-1]

# Chaikin Oscillator 구현
class ChaikinOscillator(bt.Indicator):
    lines = ('chaikin',)
    params = (('short_period', 3), ('long_period', 10))
    
    def __init__(self):
        self.adl = ADL(self.data)
        self.lines.chaikin = bt.indicators.ExponentialMovingAverage(self.adl.adl_line, period=self.p.short_period) - \
                             bt.indicators.ExponentialMovingAverage(self.adl.adl_line, period=self.p.long_period)

# Ease of Movement (EOM) 지수 구현
class EOM(bt.Indicator):
    lines = ('eom',)
    params = (('volume_factor', 1e6),)  # 거래량 스케일링 팩터
    
    def __init__(self):
        self.addminperiod(2)

    def next(self):
        high = self.data.high
        low = self.data.low
        volume = self.data.volume
        
        mid_point_move = ((high[0] + low[0]) / 2) - ((high[-1] + low[-1]) / 2)
        box_ratio = volume[0] / self.p.volume_factor / (high[0] - low[0])
        
        self.lines.eom[0] = mid_point_move / box_ratio

class OBV(bt.Indicator):
    lines = ('obv',)
    
    def __init__(self):
        self.addminperiod(1)
    
    def nextstart(self):
        # 첫 번째 데이터 포인트에서 OBV는 거래량입니다.
        self.lines.obv[0] = self.data.volume[0]
    
    def next(self):
        # 다음 데이터 포인트부터 OBV를 계산합니다.
        if self.data.close[0] > self.data.close[-1]:
            self.lines.obv[0] = self.lines.obv[-1] + self.data.volume[0]
        elif self.data.close[0] < self.data.close[-1]:
            self.lines.obv[0] = self.lines.obv[-1] - self.data.volume[0]
        else:
            self.lines.obv[0] = self.lines.obv[-1]



# 선택한 지표들
indicators = {
    "OBV": OBV, # 거래량지표
    "ADI": ADI, # 거래량지표
    "EOM": EOM, # 시장지표
    "MFI": MoneyFlowIndex, # 시장지표
    "MACD": bt.indicators.MACD, #추세지표
    "ADX": bt.indicators.ADX, #추세지표
    "RSI": bt.indicators.RSI_SMA, #탄력성지표
    "ATR": bt.indicators.AverageTrueRange #탄력성지표
    # "Stochastic": bt.indicators.Stochastic #탄력성지표
}
def fetch_indicators(ticker, start_date, end_date):
    # 데이터 로드
    data = yf.download(ticker, start=start_date, end=end_date)
    data['OpenInterest'] = 0  # backtrader에서 필요한 항목 추가

    # Backtrader 전용 데이터 피드 생성
    k200_data = bt.feeds.PandasData(dataname=data)

    class IndicatorFetcher(bt.Strategy):
        def __init__(self):
            self.indicators = {name: ind(self.data) for name, ind in indicators.items()}

    # 지표 초기화
    cerebro = bt.Cerebro(stdstats=False)  # 기본 통계자료를 포함하지 않음
    cerebro.adddata(k200_data)
    cerebro.addstrategy(IndicatorFetcher)
    
    # 백테스팅 실행
    result = cerebro.run()[0]
    
    # 결과 DataFrame 생성
    df = pd.DataFrame(index=data.index)
    for name in indicators.keys():
        df[name] = result.indicators[name].lines[0].array

    return df.dropna()

    
def save_to_csv(df, filename="test.csv"):
    # 소숫점 둘째 자리에서 반올림
    df_rounded = df.round(2)
    # CSV로 저장
    df_rounded.to_csv(filename)
    print(f"Data saved to {filename}")

# 예제 사용
ticker = "^KS11"  # KOSPI 200
start_date = "2011-02-01"
end_date = "2023-07-30"
df = fetch_indicators(ticker, start_date, end_date)
save_to_csv(df)  # 데이터를 CSV로 저장

[*********************100%***********************]  1 of 1 completed
Data saved to test.csv
